In [30]:
PNUMBER = 115792089237316195423570985008687907853269984665640564039457584007908834671663
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141  # Number of points in the field
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
GPOINT = (Gx, Gy)  # This is our generator point. Trillions of dif ones possible


def modinv(a: int, PNUMBER):
    # НАИБОЛЬШИЙ ОБЩИЙ ДЕЛИТЕЛЬ: Расширенный алгоритм Евклида / "деление" на эллиптических кривых
    lm, hm = 1, 0
    remainder = a % PNUMBER
    high = PNUMBER
    while remainder > 1:
        ratio = high // remainder
        nm = hm - lm * ratio
        new = high - remainder * ratio
        lm, remainder, hm, high = nm, new, lm, remainder
    return lm % PNUMBER


def ECadd(q, GPOINT, PNUMBER):  # Not true addition, invented for EC. Could have been called anything.
    LamAdd = ((GPOINT[1] - q[1]) * modinv(GPOINT[0] - q[0], PNUMBER)) % PNUMBER
    x = (LamAdd * LamAdd - q[0] - GPOINT[0]) % PNUMBER
    y = (LamAdd * (q[0] - x) - q[1]) % PNUMBER
    return x, y


def ECdouble(q, PNUMBER):  # This is called point doubling, also invented for EC.
    Lam = ((3 * q[0] * q[0]) * modinv(2 * q[1], PNUMBER)) % PNUMBER
    x = (Lam * Lam - 2 * q[0]) % PNUMBER
    # y = (Lam * (q[0] - x) - q[1]) % PNUMBER
    y = (x**3 + 7)**0.5 % PNUMBER
    return x, y

def ECdevide(q, prv_hex: int):
    if prv_hex == 0 or prv_hex >= N:
        raise Exception("Invalid Scalar/Private Key")
    prv_bin = str(bin(prv_hex))[2:]  # двоичная строка без начала 0b
    Q = GPOINT  # это кортеж из двух целых чисел точки генерации кривой
    for i in range(1, len(prv_bin)):
        Q = ECdouble(Q, PNUMBER)
        if prv_bin[i] == "1":
            Q = ECadd(GPOINT, Q, PNUMBER)
    return Q

def EccMultiply(prv_hex: int):  # Double & add. Not true multiplication
    if prv_hex == 0 or prv_hex >= N:
        raise Exception("Invalid Scalar/Private Key")
    prv_bin = str(bin(prv_hex))[2:]  # двоичная строка без начала 0b
    Q = GPOINT  # это кортеж из двух целых чисел точки генерации кривой
    for i in range(1, 5):
        Q = ECdouble(Q, PNUMBER)
        if prv_bin[i] == "1":
            Q = ECadd(GPOINT, Q, PNUMBER)
    return Q



def private_2_public(hex_private_key: hex):
    public_key = EccMultiply(hex_private_key)
    return hex(public_key[0])[2:], hex(public_key[1])[2:]

In [60]:
# Вариант без конечных полей, упрощенная версия

In [61]:
def add(Ax, Ay, Bx, By, times=1):
    for i in range(times):
        slope = (By - Ay) / (Bx - Ax)
        Sx = slope**2 - Ax - Bx
        Sy = slope * (Ax - Sx) - Ay
        Ax, Ay = Sx, Sy
    return Sx, Sy

def double(Ax, Ay, times=1):
    for i in range(times):
#         slope = (3 * Dx**2) / (2 * (Dx**3 + 7)**0.5)
        slope = (3 * Ax**2) / (2 * Ay)
        Dx = slope**2 - Ax - Ax
        Dy = slope * (Ax - Dx) - Ay
        Ax, Ay = Dx, Dy
    return Dx, Dy

def subtract(Ax, Ay, Bx, By, times=1):
    By = -1 * By
    for i in range(times):
        slope = (By - Ay) / (Bx - Ax)
        Sx = slope**2 - Ax - Bx
        Sy = slope * (Ax - Sx) - Ay
        Ax, Ay = Sx, Sy
    return Sx, Sy

def ECdevide(Ax, Ay, times=1):
    for i in range(times):
#         slope = (3 * Dx**2) / (2 * (Dx**3 + 7)**0.5)
        slope = (3 * Ax**2) / (2 * Ay)
        Dx = 4*Ax
        Dy = (Dx**3 + 7)**0.5
        Ax, Ay = Dx, Dy
    return Dx, Dy

def mult(Ax, Ay, prv_hex):
    Q = (Ax, Ay)
    prv_bin = str(bin(prv_hex))[2:]  # двоичная строка без начала 0b
    for i in range(1, len(prv_bin)):
        Q = double(Q[0], Q[1])
        if prv_bin[i] == "1":
            Q = add(Q[0], Q[1], Ax, Ay)
    return Q

In [62]:
Ax, Ay = 32.2,182.74
count = 1

In [63]:
Cx, Cy = double(Ax, Ay, count)
Cx, Cy

(8.033382057989513, 22.93677695354728)

In [64]:
# в результате деления координаты Cx, Cy приблизительно равны исходным Ax, Ay
ECdevide(Cx, Cy, count)

(32.13352823195805, 182.17275528759333)

In [65]:
Ax, Ay = 8.033382057989513, 22.93677695354728

In [66]:
Cx, Cy = double(Ax, Ay, count)
Cx, Cy

(1.7451840783093964, 3.6020619610941935)

In [67]:
# при следующей итерации координаты Cx, Cy отличаются значительнее от исходных Ax, Ay
ECdevide(Cx, Cy, count)

(6.9807363132375855, 18.632660136962517)

In [68]:
Ax, Ay = 1.7451840783093964, 3.6020619610941935

In [69]:
Cx, Cy = double(Ax, Ay, count)
Cx, Cy

(-1.8817790304392974, 0.9980214068383884)

In [70]:
# на третьей итерации координаты Cx, Cy "совсем далеки" от исходных Ax, Ay, тк мы еще дальше от интервала (30, +inf)
ECdevide(Cx, Cy, count)

(-7.52711612175719, (1.254093647539275e-15+20.48090352928573j))